In [ ]:
from pathlib import Path

def find_project_root(name_substring: str = "riseholme",
                      required_subpath: Path = Path("Data") / "Normal" / "Ripe",
                      start: Path = Path.cwd()) -> Path:
    """
    Durchläuft start und alle Eltern:
      1. Prüft, ob das aktuelle Verzeichnis selbst passt (Name + required_subpath).
      2. Prüft alle direkten Unterverzeichnisse (Geschwister), ob dort
         name_substring im Namen ist und required_subpath existiert.
    Gibt das gefundene Verzeichnis (Path) zurück oder wirft FileNotFoundError.
    """
    for p in [start] + list(start.parents):
        # 1) Ist p selbst unser Projekt-Root?
        if name_substring.lower() in p.name.lower() and (p / required_subpath).is_dir():
            return p

        # 2) Sonst durchsuche alle Unterordner von p nach unserem Projekt-Root
        for child in p.iterdir():
            if (child.is_dir()
                and name_substring.lower() in child.name.lower()
                and (child / required_subpath).is_dir()):
                return child

    raise FileNotFoundError(f"Kein Verzeichnis gefunden, das '{required_subpath}' enthält "
                            f"und '{name_substring}' im Namen trägt (Beginn: {start})")

# Verwende die neue Funktion
BASE_DIR = find_project_root()
print(f"✅ Projekt-Root gefunden: {BASE_DIR}")

# Jetzt kannst du weitermachen:
image_dir = BASE_DIR / 'Data' / 'Normal' / 'Ripe'
label_dir = BASE_DIR / 'labels' / 'Ripe'
label_dir.mkdir(parents=True, exist_ok=True)

class_id = 0  # Klasse 0 = "ripe"
count = 0

for img_path in image_dir.iterdir():
    if img_path.suffix.lower() in {'.jpg', '.jpeg', '.png'}:
        txt_path = label_dir / f"{img_path.stem}.txt"
        with open(txt_path, 'w') as f:
            f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
        count += 1

print(f"✅ {count} Labels erstellt in {label_dir}")


FileNotFoundError: Kein Verzeichnis gefunden, das 'Data\Normal\Ripe' enthält (Beginn: c:\Users\Maxi\Documents\Forschsem\Erdbeeren\Versuch_2\Erdbeeren_yolo\Riseholme)

datensatz splitten

In [ ]:
from pathlib import Path
from sklearn.model_selection import train_test_split
import shutil

def find_project_root(required_subpath: Path = Path("Data") / "Normal" / "Ripe",
                      start: Path = Path.cwd()) -> Path:
    """
    Durchläuft start und alle Eltern:
      1. Prüft, ob das aktuelle Verzeichnis selbst required_subpath enthält.
      2. Durchsucht alle Unterordner jedes Eltern-Verzeichnisses nach required_subpath.
    Gibt das erste gefundene Verzeichnis zurück oder wirft FileNotFoundError.
    """
    for p in [start] + list(start.parents):
        # 1) Passt p selbst?
        if (p / required_subpath).is_dir():
            return p
        # 2) Oder eine seiner Unterordner?
        for child in p.iterdir():
            if child.is_dir() and (child / required_subpath).is_dir():
                return child
    raise FileNotFoundError(f"Kein Verzeichnis gefunden, das '{required_subpath}' enthält (Beginn: {start})")

# 1) Projekt-Root ermitteln
BASE_DIR = find_project_root()
print(f"✅ Projekt-Root gefunden: {BASE_DIR}")

# 2) Quell- und Ziel-Pfade definieren
IMG_SRC   = BASE_DIR / 'Data'    / 'Normal' / 'Ripe'
LBL_SRC   = BASE_DIR / 'labels'  / 'Ripe'

IMG_TRAIN = BASE_DIR / 'images'  / 'train'
IMG_VAL   = BASE_DIR / 'images'  / 'val'
LBL_TRAIN = BASE_DIR / 'labels'  / 'train'
LBL_VAL   = BASE_DIR / 'labels'  / 'val'

# 3) Zielordner anlegen
for d in (IMG_TRAIN, IMG_VAL, LBL_TRAIN, LBL_VAL):
    d.mkdir(parents=True, exist_ok=True)

# 4) Dateinamen (Stem) sammeln und splitten
files = [f.stem for f in IMG_SRC.iterdir() if f.suffix.lower() in {'.png', '.jpg', '.jpeg'}]
train, val = train_test_split(files, test_size=0.2, random_state=42)

# 5) Kopieren
for f in train:
    # Bild-Extension automatisch prüfen
    src_img = next(IMG_SRC.glob(f"{f}.*"))
    shutil.copy(src_img, IMG_TRAIN / src_img.name)

    src_lbl = LBL_SRC / f"{f}.txt"
    shutil.copy(src_lbl, LBL_TRAIN / src_lbl.name)

for f in val:
    src_img = next(IMG_SRC.glob(f"{f}.*"))
    shutil.copy(src_img, IMG_VAL / src_img.name)

    src_lbl = LBL_SRC / f"{f}.txt"
    shutil.copy(src_lbl, LBL_VAL / src_lbl.name)

print(f"✅ Train: {len(train)} | Val: {len(val)} Dateien kopiert")


In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')             # oder yolov8s.pt, -m.pt etc.
model.train(
    data='data.yaml',                  # deine Konfig
    epochs=50,                         # je nach Bedarf
    imgsz=640,
    batch=16,
    augment=True,
    project='runs/train',              # Ausgabeordner
    name='ripe_only'                   # Lauf-Name
)
